In [18]:
import json_lines
import re
import json
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag
import string
import pandas as pd
import mtranslate


In [19]:
def penn_to_wn(tag):
    """
    Convert between the PennTreebank tags to simple Wordnet tags
    """
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

In [27]:
df = pd.DataFrame(columns=['Review', 'Topic', 'Sentiment', 'Source'])
lemmatizer = WordNetLemmatizer()

#For some reason, please returns positive sentiment because it assumes 'pleasing' but normally people just asking politely
weird_sentiment_words = ['la', 'please']

#file = json.loads(open('chi.json', encoding="utf8"))
with open('chi.json', 'rb') as f:
    for line in json_lines.reader(f):

        try:
            #Check if key exists
            sentence = line['reviewBody']
            #print(line['message'])

            print("New review")
            print('---------------------------------------')

            #Translate to English
            sentence = mtranslate.translate(sentence, 'en', 'auto')
            
            #Split a single comment into multiple sentences
            sentences = sent_tokenize(sentence)
            #print(sentences)
            
            total_sentiment = 0
            list_of_nouns = []

            #Sentiment for each sentence
            for s in sentences:

                s = s.lower()
                s = s.replace('chi fitness', ' ')
                s = s.replace('chi', ' ')
                s = re.sub(r"\byin\b", "", s)
                s = re.sub(r"\byang\b", "", s)

                print("Sentence: ", s)
                sentiment = 0.0
                #print(s)

                tokenized = word_tokenize(s)
                #print(tokenized)
                
                new_tokenized = [x for x in tokenized if x not in weird_sentiment_words]
                tokenized = new_tokenized

                #POS tag each word
                tagged = pos_tag(tokenized)
                print(tagged)

                previous_word = ""

                #Get the sentiment for each word
                for w, t in tagged:
                    
                    sentiment_added = False
                    
                    #Convert Penn Treebank POS tag to Wordnet POS tag
                    wn_tag = penn_to_wn(t)
                    
                    #If successfully converted, lemmatize using the Wordnet POS tag
                    #If not successfully converted, lemmatize without any POS tags
                    if wn_tag is None:
                        lemma = lemmatizer.lemmatize(w)
                    else:
                        lemma = lemmatizer.lemmatize(w, pos=wn_tag)
                        
                    #If can't lemmatize, set the word to be passed into SentiWordnet as the initial word, w
                    #If can lemmatize, set the word to be passed into Sentiwordnet as the lemmatized word, lemma
                    if not lemma:
                        print("Can't lemmatize: ", w)
                        final_word = w
                    else:
                        print("Lemmatized word: ", lemma)
                        final_word = lemma
                        
                    #Make cool and wow positive words
                    if (final_word in ['cool', 'wow', 'glad']) and (previous_word == 'not'):
                        print('Not is the previous word')
                        sentiment = sentiment - 0.5
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word in ['cool', 'wow', 'glad']):
                        print(final_word, " has a score of 0.5")
                        sentiment = sentiment + 0.5
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word == 'nice') and (previous_word == 'not'):
                        print('Not is the previous word')
                        sentiment = sentiment - 0.875
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word == 'nice'):
                        print(final_word, " has a score of 0.875")
                        sentiment = sentiment + 0.875
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word in ['terminate', 'stop', 'halt', 'cancel']) and (previous_word == 'not'):
                        sentiment = sentiment + 0.75
                        print("The word terminate, stop or halt is detected and the sentiment is -0.75")
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word in ['terminate', 'stop', 'halt', 'cancel']):
                        sentiment = sentiment - 0.75
                        print("The word terminate, stop or halt is detected and the sentiment is -0.75")
                        previous_word = final_word
                        sentiment_added = True
                        
                    #If the Wordnet POS tag exists, use it to get the synset of the word
                    if wn_tag is None:
                        synsets = wn.synsets(final_word)
                    else:
                        synsets = wn.synsets(final_word, pos=wn_tag)
                        
                    #If no synsets found, skip the word
                    if not synsets:
                        print("Synset not found")
                        continue
                        
                    #If the Wordnet POS tag exists, use it to get the synset of the word
                    if wn_tag is None:
                        synsets = wn.synsets(final_word)
                    else:
                        synsets = wn.synsets(final_word, pos=wn_tag)
                        
                    #If no synsets found, skip the word
                    if not synsets:
                        print("Synset not found")
                        continue
                        
                    # Take the first sense of the word which is the most common
                    synset = synsets[0]
                    swn_synset = swn.senti_synset(synset.name())
                    
                    if sentiment_added == False:
                        if final_word != 'fitness' and previous_word == 'not':      
                            print(final_word, " | ", synset, " | ", swn_synset, " | ", synset.definition())
                            sentiment = sentiment - swn_synset.pos_score() - swn_synset.neg_score()
                        elif final_word != 'fitness':
                            print(final_word, " | ", synset, " | ", swn_synset, " | ", synset.definition())
                            sentiment = sentiment + swn_synset.pos_score() - swn_synset.neg_score()
                        else:
                            print("The word fitness is not considered as a sentiment")
                    
                    previous_word = final_word

                    if t.startswith('N') or w == 'post':
                        if w != 'i' and w != 'la' and w != 'hi':
                            list_of_nouns.append(w)       
                            
                # sum greater than 0 => positive sentiment
                total_sentiment += sentiment

                    
            df2 = pd.DataFrame([[sentence, ','.join(list_of_nouns), total_sentiment, 'Facebook Review Chi Fitness']],columns=['Review', 'Topic', 'Sentiment', 'Source'])

            df = pd.concat([df2,df], ignore_index=True)
        except:
            continue

New review
---------------------------------------
Sentence:  i would like to address my concern regarding to the customer service issues to  -x fitness - starling mall management.
[('i', 'NN'), ('would', 'MD'), ('like', 'VB'), ('to', 'TO'), ('address', 'VB'), ('my', 'PRP$'), ('concern', 'NN'), ('regarding', 'VBG'), ('to', 'TO'), ('the', 'DT'), ('customer', 'NN'), ('service', 'NN'), ('issues', 'NNS'), ('to', 'TO'), ('-x', 'VB'), ('fitness', 'JJ'), ('-', ':'), ('starling', 'VBG'), ('mall', 'JJ'), ('management', 'NN'), ('.', '.')]
Lemmatized word:  i
i  |  Synset('iodine.n.01')  |  <iodine.n.01: PosScore=0.0 NegScore=0.0>  |  a nonmetallic element belonging to the halogens; used especially in medicine and photography and in dyes; occurs naturally only in combination in small quantities (as in sea water or rocks)
Lemmatized word:  would
Synset not found
Lemmatized word:  like
like  |  Synset('wish.v.02')  |  <wish.v.02: PosScore=0.125 NegScore=0.0>  |  prefer or wish to do something
Lemma

In [13]:
#df.to_json('chi_fitness_comments_sentiment.json')

In [28]:
df_filtered = df[df['Sentiment'] != 0]
print(len(df_filtered[df_filtered['Sentiment'] < 0]))
print(len(df_filtered[df_filtered['Sentiment'] > 0]))

11
8


In [29]:
df_filtered[df_filtered['Sentiment'] > 0]

,Review,Topic,Sentiment,Source
0,Good place to workout . But Im tried of pickin...,"place,workout,people,weights,bars,floor,benchs...",0.125,Facebook Review Chi Fitness
1,"I really like this place, I go to Sunway Velo...","place,velocity,house,person,place,staff,job,guys",1.625,Facebook Review Chi Fitness
3,"Well for me, I go to Damen though it's a much ...","gym,terms,size,friendliness,members,staff,year...",2.750,Facebook Review Chi Fitness
6,Chi-Fitness IKON is one of the best fitness ce...,"ikon,fitness,center,lots,light,training,space,...",4.750,Facebook Review Chi Fitness
7,please becareful of leonardo from queensbay ma...,"leonardo,mall,manager,customer,time,staff,guy,...",0.500,Facebook Review Chi Fitness
10,I went to your gym at Bangsar South on 4th Aug...,"gym,south,press,bent,exercise,bent,exercise,st...",4.125,Facebook Review Chi Fitness
13,Hi! I have been a member of CHI Fitness Starli...,"member,month,day,self,gym,membership,pt,sessio...",1.750,Facebook Review Chi Fitness
14,Chi Fitness Damen is my stress reliever playgr...,"stress,reliever,playground,ambiance,staff,inst...",4.125,Facebook Review Chi Fitness


In [30]:
df_filtered[df_filtered['Sentiment'] < 0]

,Review,Topic,Sentiment,Source
2,Was absolutely bad! When I had the agreement w...,"agreement,mention,contract,account,account,fre...",-2.625,Facebook Review Chi Fitness
4,Hi .. I wish to feedback Chi Fitness at Queens...,"mall,experience,speakers,studio,years,solution...",-0.375,Facebook Review Chi Fitness
5,"Hi Chi Fitness, I wish to lodge a complaint on...","complaint,manager,branch,membership,months,acc...",-1.250,Facebook Review Chi Fitness
8,"Hi, today i went to Atria mall CHI fitness for...","today,mall,trial,friend,front,girl,staff,today...",-3.125,Facebook Review Chi Fitness
9,Nowdays management were duin unstoppable nons...,"management,things,business,profit,gym,shit,man...",-0.500,Facebook Review Chi Fitness
11,I've been a CHI member since 2014 and been cha...,"member,member,atria,esm,management,account,cus...",-1.000,Facebook Review Chi Fitness
12,"Same here, I'm totally disappointed I have alr...","contract,atria,month,month,charge,credit,card,...",-0.125,Facebook Review Chi Fitness
15,I really disappointed and fed up for what the ...,"services,mall,membership,august,september,term...",-1.500,Facebook Review Chi Fitness
16,The hot water shower in the ladies changing ro...,"water,shower,ladies,room,nexus,jan,enquiry,sta...",-0.250,Facebook Review Chi Fitness
17,I am extremely dissapointed. My gf and I both ...,"promotion,day,gym,centre,manager,holiday,day,g...",-4.375,Facebook Review Chi Fitness


In [31]:
df_filtered.to_json('chifitnessreviewsentiment.json', orient='records')

In [ ]:
test = df_filtered[df_filtered['Sentiment'] > 0]
test[test['Topic'].str.contains('obstacles')]